# Fold first, ask later: structure-informed function annotation of *Pseudomonas* phage proteins
## Pipeline stage 2: annotation

### Stage 2.X: annotation - investigating a future-proof annotation classification method

**Goals:**
* Identify a feasible, representative subset of protein annotations to train the classifier on.
* Label using the current REGEX method.
* Manually correct labels to desired label.

**Requires:**
* in folder Desktop/phage_annotation_input:
    * FoldSeek search database header files af50m_2024mar_h (FoldSeek 'Alphafold/UniProt50-minimal' database v.4) 

**Generates:**

#### General settings, imports, variables and environments

Conda environment: ffal_annotate

Created with `conda create -n ffal_annotate`. Then installed `jupyter notebook`, `pandas` through conda & `seaborn`, `scipy`, `matplotlib-venn`, `UpSetPlot`, `biopython` through pip (commands executed in notebook Code_annotation_background.ipynb).

In [ ]:
!pip install seaborn

In [ ]:
!pip install scipy

In [ ]:
!pip install matplotlib-venn

In [ ]:
!pip install UpSetPlot

In [ ]:
!pip install biopython

In [1]:
!conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: win-64
# created-by: conda 24.11.3
@EXPLICIT
https://conda.anaconda.org/conda-forge/noarch/ca-certificates-2025.6.15-h4c7d964_0.conda
https://conda.anaconda.org/conda-forge/win-64/intel-openmp-2024.2.1-h57928b3_1083.conda
https://conda.anaconda.org/conda-forge/noarch/python_abi-3.13-7_cp313.conda
https://conda.anaconda.org/conda-forge/noarch/tzdata-2025b-h78e105d_0.conda
https://conda.anaconda.org/conda-forge/win-64/ucrt-10.0.22621.0-h57928b3_1.conda
https://conda.anaconda.org/conda-forge/win-64/winpty-0.4.3-4.tar.bz2
https://conda.anaconda.org/conda-forge/win-64/libwinpthread-12.0.0.r4.gg4f2fc60ca-h57928b3_9.conda
https://conda.anaconda.org/conda-forge/win-64/vc14_runtime-14.44.35208-h818238b_26.conda
https://conda.anaconda.org/conda-forge/win-64/vc-14.3-h41ae7f8_26.conda
https://conda.anaconda.org/conda-forge/win-64/vs2015_runtime-14.44.35208-h38c0c73_26.conda
https://

In [2]:
# imports - just kept from annotation notebook, probably don't use all of those here
import csv
import json
import os
import random
import re
import requests
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import Bio

from Bio.PDB import *
from json.decoder import JSONDecodeError
from scipy.stats import mannwhitneyu
from upsetplot import plot

In [3]:
# clear reference to different directories
pipeline_annot_dir = os.getcwd()
pipeline_search_dir = os.path.join(pipeline_annot_dir, os.pardir, "1_dataset")
master_dir = os.path.abspath(os.path.join(pipeline_annot_dir, os.pardir, os.pardir))
data_dir = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop', 'phage_annotation_input')
struct_out_dir = os.path.join(os.path.join(os.environ["USERPROFILE"]), "Desktop", "phage_annotation_input", "structure_prediction") 
comp_out_dir = os.path.join(os.path.join(os.environ["USERPROFILE"]), "Desktop", "phage_annotation_input", "structure_comparison") 

#### Goal 1 : Identify a feasible, representative subset of protein annotations to train the classifier on.

Let's try and see how much unique annotations are in the reduced AlphaFold database:

In [4]:
 # read in the protein annotation data - AF50m database
af50mid_descr_db = {}
with open(os.path.join(data_dir, "af50m_2024mar_h")) as fp:
    for line in fp:
        if len(line.strip()) != 0 :
            af_id = line.split()[0].removeprefix("\x00")
            descr = " ".join(line.split()[1:])
            af50mid_descr_db[af_id] = descr

In [5]:
# create an overview of the unique annotations in the AFdb
unique_ann = list(set(af50mid_descr_db.values()))

In [6]:
len(unique_ann) # 1.6M, which is a lot - let's reduce this 

1618820

If we look at all the unique entries, we see there's 1.6 million protein entries. It is not feasible to manually label those right now. Let's try and focus on the subset that we actually found during the annotation process:

In [7]:
# reading in the map between UniProt ID and UniProt protein names
    # since I did not use DictWriter properly, we have created too large fields for DictReader to reader, which I first have to accomodate here
max_int = sys.maxsize
while True:
    # decrease the max_int value by factor 10 
    # as long as the OverflowError occurs.
    try:
        csv.field_size_limit(max_int)
        break
    except OverflowError:
        max_int = int(max_int/10)
csv.field_size_limit(max_int)
    # actually reading in the data
with open(os.path.join(data_dir, "dict_UniProtID_function.csv"), newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dict_UniProtID_function = row

In [8]:
# switching the dictionary keys from AFdb identifiers to UniProt identifiers
af50m_up_descr = {key.split("-")[1]: value for key, value in af50mid_descr_db.items() if "AF-" in key}

In [9]:
None in af50m_up_descr.values()

False

In [10]:
# taking the subset for which we fetched information from UniProt
af50mid_subset = {key: value for key, value in af50m_up_descr.items() if key in dict_UniProtID_function.keys()}

In [11]:
None in af50mid_subset.values()

False

In [12]:
# getting the unique annotations out of it
unique_ann_sub = list(set(af50mid_subset.values()))

In [13]:
len(unique_ann_sub)

22103

#### Goal 2 : Label protein annotations using the current REGEX method.

Let's now label these protein annotations, using the existing REGEXes:

For the AlphaFold database, three rounds of manual inspection of annotations led to the following REGEXes for filtering out uninformative annotations, in the context of phage proteins:

In [14]:
# final filter lists
hypothetical_af_search = ["whole genome shotgun sequence"] 
hypothetical_af_match = ["unnamed product", 
                         r"genome assembly, chromosome: [\w+]+", r"str. [\w+]+[\d+]+", 
                         "protein of unknwon function", 
                         "conserved domain protein, histidine-rich", r"lin[\d+]+ protein", r"orf[\w+]+( domain-containing)?( protein)?", 
                         "phage related-protein", "phage d3 protein", 
                         r"dna, contig(: [\w+]+)?", r"uncharacterized protein conserved in bacteria(, prophage-related)?", r"gene [\d+]+ protein",
                         r"gifsy-[\d+] prophage protein", r"(hypothetical )?genomic island protein",
                         r"(putative )?(conserved )?(\([a-z ]+\) )?((uncharacterized)|(hypothet(h)?ical)|(unannotated))( conserved)?( protein)?(, (pro)?phage-related)?(, isoform [AB])?",
                         r"(predicted|unannotated|uncharacterized|hypothetical|putative( |_)?)?(expressed)?(conserved)?((mobile element)|(integron protein cassette))?(( )?domain)?(constituent)?([\d+]+)?( |_)protein(_-_conserved)?", 
                         r"((hypothetical|putative|probable|conserved|uncharacterized) )?((hypothetical|putative|probable|conserved|uncharacterized) )?(bacterio|pro)?(phage)((-| )(like|related|associated))?( (hypothetical|putative|probable|conserved|uncharacterized))?( protein)?(,? [g]?p[\d+]+\b)?(, family)?(, putative)?",
                         r"hk97( family)?( phage)? protein", r"putative similar to (bacterio)?phage protein",
                         r"(putative )?uncharacterized protein(?! duf[\d+]+)( \w+)?", 
                         r"(phage( |-))?(protein )?(putative )?[g]?p[\d+]+\b(-like)?( domain(-containing)?)?(( |-)family)?( protein)?"
                        ]

In [15]:
# functions to check whether description is in the list of hypothetical protein annotations for AFdb annotations
def is_hypothetical_af(descr):
    hypothetical_label = False
    for filter_descr_s in hypothetical_af_search:
        if re.search(filter_descr_s, descr.strip().lower()) != None:
            hypothetical_label = True
    for filter_descr_m in hypothetical_af_match:
        if re.fullmatch(filter_descr_m, descr.strip().lower()) != None:
            hypothetical_label = True
    return hypothetical_label

In [16]:
# set of filters to identify low informative annotations
low_af_search = [r"duf[0-9]+", "br0599"] 
low_af_match = [r"uncharacterized (mitochondrial )?protein ((xf_1581/xf_1686)|(atmg00810-like))",
                  r"uncharacterized( membrane)? protein ([\w+]+ )?\((upf)[\d+]+( family\))", r"cson[\d+]+ protein", 
                  "putative phage-related exported protein",
                  r"((hypothetical|predicted|putative) )?(conserved )?((pro)?phage )?(cytosolic|membrane|secreted|exported|cytoplasmic)( associated)?( phage)? protein(, putative)?"
                 ]

In [17]:
# function to test whether an annotation is to be considered low informative
def is_low_af(descr):
    low_label = False
    for filter_descr_s in low_af_search:
        if re.search(filter_descr_s, descr.strip().lower()) != None:
            low_label = True
    for filter_descr_m in low_af_match:
        if re.fullmatch(filter_descr_m, descr.strip().lower()) != None:
            low_label = True
    return low_label

In [19]:
file = open("AF50m_annotations_REGEXlabelled.tsv", "w")
file.write(f"protein_annotation\tregex_label\n")
for annotation in random.sample(unique_ann_sub, 22103):
    label = ""
    if is_hypothetical_af(annotation) == True:
        label = "uninformative"
    elif is_low_af(annotation) == True:
        label = "low informative"
    else:
        label = "proper"
    file.write(f"{annotation}\t{label}\n")
file.close()

#### Goal 3 : Manually correct labels to desired label.

Let's now manually add a third column to this table labeling the proteins with the desired label. 